In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import *
%matplotlib inline

In [3]:
heartAtk = pd.read_csv("./heart_attack_prediction_dataset.csv")
heartAtk.columns

Index(['Patient ID', 'Age', 'Sex', 'Cholesterol', 'Blood Pressure',
       'Heart Rate', 'Diabetes', 'Family History', 'Smoking', 'Obesity',
       'Alcohol Consumption', 'Exercise Hours Per Week', 'Diet',
       'Previous Heart Problems', 'Medication Use', 'Stress Level',
       'Sedentary Hours Per Day', 'Income', 'BMI', 'Triglycerides',
       'Physical Activity Days Per Week', 'Sleep Hours Per Day', 'Country',
       'Continent', 'Hemisphere', 'Heart Attack Risk'],
      dtype='object')

In [4]:
heartAtk.shape

(8763, 26)

In [5]:
heartAtk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8763 entries, 0 to 8762
Data columns (total 26 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Patient ID                       8763 non-null   object 
 1   Age                              8763 non-null   int64  
 2   Sex                              8763 non-null   object 
 3   Cholesterol                      8763 non-null   int64  
 4   Blood Pressure                   8763 non-null   object 
 5   Heart Rate                       8763 non-null   int64  
 6   Diabetes                         8763 non-null   int64  
 7   Family History                   8763 non-null   int64  
 8   Smoking                          8763 non-null   int64  
 9   Obesity                          8763 non-null   int64  
 10  Alcohol Consumption              8763 non-null   int64  
 11  Exercise Hours Per Week          8763 non-null   float64
 12  Diet                

In [6]:
heartAtk.head()

,Patient ID,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,Obesity,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
0,BMW7812,67,Male,208,158/88,72,0,0,1,0,...,6.615001,261404,31.251233,286,0,6,Argentina,South America,Southern Hemisphere,0
1,CZE1114,21,Male,389,165/93,98,1,1,1,1,...,4.963459,285768,27.194973,235,1,7,Canada,North America,Northern Hemisphere,0
2,BNI9906,21,Female,324,174/99,72,1,0,0,0,...,9.463426,235282,28.176571,587,4,4,France,Europe,Northern Hemisphere,0
3,JLN3497,84,Male,383,163/100,73,1,1,1,0,...,7.648981,125640,36.464704,378,3,4,Canada,North America,Northern Hemisphere,0
4,GFO8847,66,Male,318,91/88,93,1,1,1,1,...,1.514821,160555,21.809144,231,1,5,Thailand,Asia,Northern Hemisphere,0


In [7]:
columns_to_drop=['Patient ID','Hemisphere','Continent']
heartAtk_clean = heartAtk.drop(columns_to_drop,axis=1)

In [8]:
X = heartAtk_clean.drop(['Heart Attack Risk'],axis=1)
y = heartAtk_clean['Heart Attack Risk'].to_frame()
y

,Heart Attack Risk
0,0
1,0
2,0
3,0
4,0
...,...
8758,0
8759,0
8760,1
8761,0


In [9]:
heartAtk_clean.shape

(8763, 23)

In [10]:
heartAtk_clean.describe()

,Age,Cholesterol,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,Exercise Hours Per Week,Previous Heart Problems,Medication Use,Stress Level,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Heart Attack Risk
count,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000,8763.000000
mean,53.707977,259.877211,75.021682,0.652288,0.492982,0.896839,0.501426,0.598083,10.014284,0.495835,0.498345,5.469702,5.993690,158263.181901,28.891446,417.677051,3.489672,7.023508,0.358211
std,21.249509,80.863276,20.550948,0.476271,0.499979,0.304186,0.500026,0.490313,5.783745,0.500011,0.500026,2.859622,3.466359,80575.190806,6.319181,223.748137,2.282687,1.988473,0.479502
min,18.000000,120.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002442,0.000000,0.000000,1.000000,0.001263,20062.000000,18.002337,30.000000,0.000000,4.000000,0.000000
25%,35.000000,192.000000,57.000000,0.000000,0.000000,1.000000,0.000000,0.000000,4.981579,0.000000,0.000000,3.000000,2.998794,88310.000000,23.422985,225.500000,2.000000,5.000000,0.000000
50%,54.000000,259.000000,75.000000,1.000000,0.000000,1.000000,1.000000,1.000000,10.069559,0.000000,0.000000,5.000000,5.933622,157866.000000,28.768999,417.000000,3.000000,7.000000,0.000000
75%,72.000000,330.000000,93.000000,1.000000,1.000000,1.000000,1.000000,1.000000,15.050018,1.000000,1.000000,8.000000,9.019124,227749.000000,34.324594,612.000000,5.000000,9.000000,1.000000
max,90.000000,400.000000,110.000000,1.000000,1.000000,1.000000,1.000000,1.000000,19.998709,1.000000,1.000000,10.000000,11.999313,299954.000000,39.997211,800.000000,7.000000,10.000000,1.000000


In [11]:
heartAtk_clean.isna().any()

Age                                False
Sex                                False
Cholesterol                        False
Blood Pressure                     False
Heart Rate                         False
Diabetes                           False
Family History                     False
Smoking                            False
Obesity                            False
Alcohol Consumption                False
Exercise Hours Per Week            False
Diet                               False
Previous Heart Problems            False
Medication Use                     False
Stress Level                       False
Sedentary Hours Per Day            False
Income                             False
BMI                                False
Triglycerides                      False
Physical Activity Days Per Week    False
Sleep Hours Per Day                False
Country                            False
Heart Attack Risk                  False
dtype: bool

In [12]:
heartAtk_clean.duplicated().sum()
heartAtk_clean.drop_duplicates(inplace=True)
heartAtk_clean.head()

,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,...,Medication Use,Stress Level,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Heart Attack Risk
0,67,Male,208,158/88,72,0,0,1,0,0,...,0,9,6.615001,261404,31.251233,286,0,6,Argentina,0
1,21,Male,389,165/93,98,1,1,1,1,1,...,0,1,4.963459,285768,27.194973,235,1,7,Canada,0
2,21,Female,324,174/99,72,1,0,0,0,0,...,1,9,9.463426,235282,28.176571,587,4,4,France,0
3,84,Male,383,163/100,73,1,1,1,0,1,...,0,9,7.648981,125640,36.464704,378,3,4,Canada,0
4,66,Male,318,91/88,93,1,1,1,1,0,...,0,6,1.514821,160555,21.809144,231,1,5,Thailand,0


Blood Pressure Values by category
Category/Systolic/Diastolic
Normal : less than 120 ; less than 80

Elevated : 120-129 ; less than 80

High Blood Pressure (Hypertension) Stage 1 : 130-139 ; 80-89

High Blood Pressure (Hypertension) Stage 2 : 140 or higher ; 90 or higher

Relevant age groups: 18-39 ; 40-59; 60+

In [16]:
labels = ['18-39', '40-59', '60+']  # Adjust the labels
df = pd.DataFrame(heartAtk_clean)
bins = [18, 39, 59, 110]  # Adjust the bins
df['Age Group'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False)
grouped = df.groupby('Age Group')
age_group_1839 = grouped.get_group('18-39')
age_group_4059 = grouped.get_group('40-59')
age_group_60over = grouped.get_group('60+')
print(age_group_1839)



      Age     Sex  Cholesterol Blood Pressure  Heart Rate  Diabetes  \
1      21    Male          389         165/93          98         1   
2      21  Female          324         174/99          72         1   
8      20    Male          145        144/105          68         1   
17     38    Male          166         120/74          56         1   
24     36    Male          133         161/90          97         1   
...   ...     ...          ...            ...         ...       ...   
8755   25    Male          307         137/94          78         0   
8756   22    Male          347         115/87         108         0   
8759   28  Female          120        157/102          73         1   
8761   36    Male          178         119/67          60         1   
8762   25  Female          356         138/67          75         1   

      Family History  Smoking  Obesity  Alcohol Consumption  ...  \
1                  1        1        1                    1  ...   
2          

/tmp/ipykernel_4363/3165133153.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby('Age Group')


In [17]:

print(age_group_4059)

      Age     Sex  Cholesterol Blood Pressure  Heart Rate  Diabetes  \
5      54  Female          297         172/86          48         1   
9      43  Female          248         160/70          55         0   
18     50  Female          303        120/100         104         1   
21     45    Male          294         130/84          66         0   
22     50    Male          359         175/60          97         0   
...   ...     ...          ...            ...         ...       ...   
8729   40    Male          383         158/70          51         1   
8732   43    Male          370         97/105          45         1   
8735   43    Male          128        107/105          94         1   
8741   45  Female          166          98/60          63         0   
8760   47    Male          250         161/75         105         0   

      Family History  Smoking  Obesity  Alcohol Consumption  ...  \
5                  1        1        0                    1  ...   
9          

In [18]:

print(age_group_60over)

      Age     Sex  Cholesterol Blood Pressure  Heart Rate  Diabetes  \
0      67    Male          208         158/88          72         0   
3      84    Male          383        163/100          73         1   
4      66    Male          318          91/88          93         1   
6      90    Male          358         102/73          84         0   
7      84    Male          220         131/68         107         0   
...   ...     ...          ...            ...         ...       ...   
8752   65    Male          150         152/99         106         1   
8753   82    Male          311        126/108          87         0   
8754   80  Female          383         153/96          91         1   
8757   59  Female          378          93/78          99         0   
8758   60    Male          121          94/76          61         1   

      Family History  Smoking  Obesity  Alcohol Consumption  ...  \
0                  0        1        0                    0  ...   
3          